In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
import os
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier

In [3]:
X = pd.read_csv('train.csv', index_col='PassengerId')
X_test = pd.read_csv('test.csv', index_col = 'PassengerId')



X.drop(X[X['Cabin'].str[0] == 'T'].index, inplace=True)
y = X.Survived
X['Cabin_Char'] = X['Cabin'].str[0]

X_test['Cabin_Char'] = X_test['Cabin'].str[0]


X = X.drop(columns = ['Survived', 'Name', 'Cabin', 'Ticket'], axis = 1)
X_test = X_test.drop(columns= ['Name', 'Cabin', 'Ticket'], axis = 1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.88, test_size=0.12,random_state=0)

X_train = pd.get_dummies(X_train_full)
X_valid = pd.get_dummies(X_valid_full)
X_test = pd.get_dummies(X_test)


print("Setup Complete")

Setup Complete


In [10]:
model = XGBClassifier(random_state = 0, n_estimators = 100, learning_rate = 0.01, early_stopping_rounds=8,max_depth=12,colsample_bytree = 0.96, eval_metric = 'mae') 
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
predictions = np.array(model.predict(X_valid))
mae = mean_absolute_error(predictions, y_valid)
print(mae)

0.1308411214953271


In [126]:
estimator = []

estimator.append(('RandomForest', RandomForestClassifier(n_estimators = 170, max_depth=6, random_state = 0) ))
estimator.append(('HistGradient', HistGradientBoostingClassifier(learning_rate=0.04, max_iter=120, max_leaf_nodes=7, max_depth=15, random_state = 0) ))
estimator.append(('XGB', XGBClassifier(random_state = 0, n_estimators = 200, learning_rate = 0.01,max_depth=10,colsample_bytree = 0.96)))


ensemble = VotingClassifier(estimators=estimator, voting='hard')

ensemble.fit(X_train, y_train)

predictions = np.array(ensemble.predict(X_valid))
mae = mean_absolute_error(predictions, y_valid)
print(mae)

0.14018691588785046


In [127]:
preds_test = ensemble.predict(X_test) # Your code here

output = pd.DataFrame({'PassengerId': X_test.index,
                       'Survived': preds_test})
output.to_csv('submission.csv', index=False)